In [1]:
from paths import Paths

ModuleNotFoundError: No module named 'paths'

In [6]:
from pyspark.sql import SparkSession
app_name = 'interactions'
# spark = SparkSession.builder.appName(app_name).getOrCreate()
spark = SparkSession.builder.config("spark.sql.debug.maxToStringFields", 1000).getOrCreate()

data_path = Paths.data


books_df = spark.read.parquet(f"{data_path}books_sample.parquet")
interactions_df = spark.read.csv(f"{data_path}goodreads_interactions.csv", header=True, inferSchema=True)
books_df.createOrReplaceTempView("books_view") # register the books_df as a temporary view
joined_df = interactions_df.hint("broadcast", "books_view").join(books_df, on="book_id", how="inner") # pass the name of the view to the hint method


23/10/18 23:23:23 WARN HintErrorLogger: Count not find relation 'books_view' specified in hint 'broadcast(books_view)'.


In [9]:
# show first 10 rows
joined_df.show(100)

+-------+-------+-------+------+-----------+----+--------------------+--------------+------------+--------------------+--------------------+---------+--------------------+--------+----------+-------------+-----------+-------------+--------------------+---------+--------------------+---------------+-----------------+----------------+--------------------+-------------+----------------+--------------------+------------------+--------------------+--------------------+--------------------+--------+
|book_id|user_id|is_read|rating|is_reviewed|asin|             authors|average_rating|country_code|         description| edition_information|   format|           image_url|is_ebook|      isbn|       isbn13|kindle_asin|language_code|                link|num_pages|     popular_shelves|publication_day|publication_month|publication_year|           publisher|ratings_count|          series|       similar_books|text_reviews_count|               title|title_without_series|                 url| work_id|
+-

In [7]:
from pyspark.sql.functions import col, avg, count, desc, first

target_book_id = "948" # change this to the book id of the target book
target_book = joined_df.filter(joined_df.book_id == target_book_id).first() # get the target book row
target_shelf = target_book.popular_shelves[0].name # get the name of the most popular shelf for the target book
target_pages = int(target_book.num_pages) # get the number of pages for the target book
target_users = joined_df.filter((joined_df.book_id == target_book_id) & (joined_df.rating >= 4)).select("user_id").distinct() # get the distinct user ids who rated the target book at least 4

recommendations_df = joined_df.filter(
    (joined_df.popular_shelves[0].name == target_shelf) & # filter by popular shelf name
    (joined_df.num_pages.between(target_pages * 0.9, target_pages * 1.1)) & # filter by number of pages within 10% difference
    (joined_df.user_id.isin(target_users.user_id)) & # filter by user ids who rated the target book well
    (joined_df.rating >= 4) & # filter by rating of at least 4
    (joined_df.book_id != target_book_id) # exclude the target book itself
).groupBy("book_id").agg( # group by book id and aggregate by
    first("title").alias("title"), # getting the first title
    first("authors").alias("authors"), # getting the first authors
    avg("rating").alias("avg_rating"), # computing the average rating
    count("user_id").alias("num_users") # counting the number of users
).orderBy(desc("avg_rating"), desc("num_users")) # ordering by average rating and number of users in descending order

recommendations_df.show(10)


AttributeError: 'NoneType' object has no attribute 'popular_shelves'